<a href="https://colab.research.google.com/github/JJOK97/core-project-study/blob/eh%2Ftext_mining/%EB%84%A4%EC%9D%B4%EB%B2%84%EC%98%81%ED%99%94%ED%8F%89%EC%A0%90%EB%A6%AC%EB%B7%B0_%23TF-IDF%20Vectorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 목표 설정
- 네이버 영화 리뷰 데이터셋을 이용해서 긍정/부정을 판별하는 분류기를 만들어보자.
- TF-IDF 방법을 이용해서 토큰화를 진행해보자.
- Konlpy 한국어 형태소 분석기를 이용해보자.
- 단어별 긍/부정 정보를 시각화

In [ ]:
# 내 경로 설정
!pwd

/content/drive/MyDrive/Colab Notebooks/SW 엔지니어


In [ ]:
from google.colab  import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#소스코드가 있는 경로로 위치를 변경
%cd /content/drive/MyDrive/Colab Notebooks/SW 엔지니어

/content/drive/MyDrive/Colab Notebooks/SW 엔지니어


In [ ]:
#필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('./data/ratings_train.csv', encoding = 'utf-8')

In [ ]:
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
#test 데이터 불러오기
test = pd.read_csv('./data/ratings_test.csv', encoding = 'utf-8')

In [ ]:
test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
#train df의 대한 정보를 확인해보자

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
#test df도 확인
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [ ]:
#결측치 확인
train[train['document'].isnull()]

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [ ]:
# .dropna : 결측치가 있는 행을 삭제해주는 함수
train.dropna(inplace = True)
test.dropna(inplace = True)

In [ ]:
train.info() #결측치 꼭 없애기!

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [ ]:
train #X(문제)는 document, Y(정답)는 label --> 지도학습

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
#학습데이터의 정답데이터
train['label']

,label
0,0
1,1
2,0
3,0
4,1
...,...
149995,0
149996,1
149997,0
149998,1


In [ ]:
#학습데이터의 문제데이터
train['document']

,document
0,아 더빙.. 진짜 짜증나네요 목소리
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
...,...
149995,인간이 문제지.. 소는 뭔죄인가..
149996,평점이 너무 낮아서...
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상


In [ ]:
#평가데이터의 문제데이터
test['document']

,document
0,굳 ㅋ
1,GDNTOPCLASSINTHECLUB
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네


In [ ]:
#train/ test 데이터셋에서 문제데이터/ 정답데이터 지정

#x : 문제
#y : 정답
#train : 훈련
#test : 평가(테스트)

#문제 데이터
x_train = train['document']
x_test = test['document']

#정답 데이터
y_train = train['label']
y_test = test['label']

In [ ]:
x_train.shape, x_test.shape,y_train.shape, y_test.shape

((149995,), (49997,), (149995,), (49997,))

### 토큰화
**vectorizer**
- vectorizer는 대상을 벡터화시켜주는 도구
- 벡터화 : 텍스트나 이미지처럼 '비수치적'인 데이터를 숫자 배열(벡터)로 바꾸는 과정
- 기계 학습 분야에서 주로 텍스트 데이터를 분석하기 위해서
- 여러 머신러닝/ 딥러닝 모델을 학습시키기 위해서는 텍스트나 다른 형태의 데이터를 벡터화/수치화할 필요가 있다.


### BOW(Bag of Word) - 단어가방
- 문서를 벡터로 변환하는 가장 기본적인 방법
- 문서 내의 모든 단어를 모아서 가방 안에 넣고, 어떤 단어가   
들어 있는지 리스트/딕셔너리 형태로 '순서-숫자'로 나타내는 방법


In [ ]:
# TF-IDF 기법 사용
from sklearn.feature_extraction.text import TfidfVectorizer


In [169]:
X_train[0:3]

,document
0,아 더빙.. 진짜 짜증나네요 목소리
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다


In [170]:
#TF-IDF 벡터라이저 사용

#객체 생성
tfidf = TfidfVectorizer()

#단어 사전 구축
tfidf.fit(X_train[0:3])


TfidfVectorizer()

In [ ]:
#분리된 단어로 만든 단어사전
tfidf.vocabulary_

{'더빙': 2,
 '진짜': 6,
 '짜증나네요': 7,
 '목소리': 3,
 '포스터보고': 9,
 '초딩영화줄': 8,
 '오버연기조차': 5,
 '가볍지': 0,
 '않구나': 4,
 '너무재밓었다그래서보는것을추천한다': 1}

In [ ]:
temp = TfidfVectorizer() #벡터화 객체화

sugar = '설탕국수를냠냠쩝쩝너무맛있게먹었다냠냠쩝쩝'
boy = 'i am a boy'
boy2 = '나는 소년이다.'

#단어사전 구축
temp.fit([sugar, boy, boy2])

temp.vocabulary_

{'설탕국수를냠냠쩝쩝너무맛있게먹었다냠냠쩝쩝': 3, 'am': 0, 'boy': 1, '나는': 2, '소년이다': 4}

- TF-IDF Vectorizer는 기본적으로 어절 단위(문장을 이루는 토막)로 끊어주기 때문에 한국어 특성상 파악이 어렵다.

- 더 작은 단위인 형태소별로 나눠서 파악해야 한다.

## 한국어 형태소 분석기 사용
- 한국어 특성에 맞춰 개발된 분석기 사용

In [ ]:
#라이브러리 설치
!pip install konlpy

In [ ]:
#형태소 분석기 불러오기
from konlpy.tag import Kkma

## Kkma(꼬꼬마)
- 서울대학교 IDS연구실 개발


In [ ]:
# 꼬꼬마 객체 생성
kkma = Kkma()

In [ ]:
sugar

'설탕국수를냠냠쩝쩝너무맛있게먹었다냠냠쩝쩝'

In [ ]:
kkma.nouns(sugar)

['설탕', '국수']

### Kkma에서 사용하는 대표적 함수
- nouns() : 명사 추출 함수
- morphs() : 형태소 추출
- pos(): 형태소 추출 + 품사 태그


In [190]:
def myTokenizer2(text):
    d = pd.DataFrame(kkma.pos(text), columns=['morph', 'tag'])
    d.set_index('tag', inplace=True)

    # 존재하는 품사만 추출 (방어적 필터링)
    wanted_tags = ['NNG', 'NNP']
    valid_tags = [tag for tag in wanted_tags if tag in d.index]

    return d.loc[valid_tags].morph.tolist()


In [191]:
X_train

,document
0,아 더빙.. 진짜 짜증나네요 목소리
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
...,...
149995,인간이 문제지.. 소는 뭔죄인가..
149996,평점이 너무 낮아서...
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상


In [192]:
# TF-IDF Vectorizer에 연결
tfidf_pos = TfidfVectorizer(tokenizer=myTokenizer2)

# 적용/학습
tfidf_pos.fit(X_train[0:3])

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function myTokenizer at 0x7dfffdb3f920>)

In [193]:
#단어사전 확인
tfidf_nouns.vocabulary_

{'더빙': 0,
 '목소리': 2,
 '흠': 17,
 '포스터': 15,
 '포스터보고': 16,
 '보고': 4,
 '초': 12,
 '초딩영화줄': 13,
 '딩': 1,
 '영화': 6,
 '줄': 11,
 '오버': 7,
 '오버연기': 8,
 '연기': 5,
 '재': 9,
 '재밓': 10,
 '밓': 3,
 '추천': 14}

In [194]:

# 실제 데이터를 이용해서 단어사전을 구축해보자
tfidf_final = TfidfVectorizer(tokenizer=myTokenizer2)
# 10000개의 데이터를 이용해서 단어사전 구축을 진행
tfidf_final.fit(X_train[:1000])

TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x7dfff9b847c0>)

In [ ]:
# transform : 텍스트데이터를 벡터화
# 실제 문장을 잘게 쪼개서 수치값으로 바꿔주자.

X_train_text = tfidf_final.transform(X_train[0:1000])
X_test_text = tfidf_final.transform(X_test)

### 모델링 단계
- 머신러닝 모델을 이용해서 텍스트 분석을 진행

In [175]:
# Logistic 회귀 모형 import
from sklearn.linear_model import LogisticRegression

In [ ]:
# 로지스틱 회귀 객체 생성
logi = LogisticRegression()

In [184]:
#로지스틱 회귀모형 학습
logi.fit(X_train_text, y_train[0:1000])

NameError: name 'logi' is not defined

In [ ]:
#10000개로 단어사전 구축
X_test.

In [ ]:
X_test